In [1]:
# IMPORTATIONS
import json
import logging
import sys
import os
from typing import List
from degiro_connector.trading.api import API as TradingAPI
from degiro_connector.trading.models.trading_pb2 import Credentials, ProductSearch, ProductsInfo
import shelve
from degiro_connector.quotecast.api import API as QuotecastAPI
from degiro_connector.quotecast.actions.action_get_chart import ChartHelper
from degiro_connector.quotecast.models.quotecast_pb2 import Chart
import pandas as pd
from datetime import datetime
import traceback
import numpy as np
from scipy.interpolate import interp1d
import re
from multiprocessing import  Pool
from functools import partial
import yfinance as yf
import threading, time, random
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import math
import itertools

class cachedApi:
    def __init__(self, file:str, credentials=Credentials):
        self.__db = shelve.open(file)
        self.__trading_api = TradingAPI(credentials=credentials)
        self.__user_token = None
        self.__quotecast_api = None
        self.mutex = threading.Lock()
        
    def logout(self):
        self.__trading_api.logout()
    
    def cache_get(self, k):
        r = None
        self.mutex.acquire()
        try:
            r = self.__db[k]
        except:
            None
        self.mutex.release()
        return r

    def cache_set(self, k,v):
        self.mutex.acquire()
        self.__db[k] = v
        self.mutex.release()
    
    def get_config(self):
        return self.__trading_api.credentials

    def get_config(self,**kwargs):
        k = 'get_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_config(**kwargs)
            self.cache_set(k,r)
        #print(r)
        self.__user_token = r['clientId']
        #print(f"token:{self.__user_token}")
        return r

    def get_client_details(self,**kwargs):
        k = 'get_client_details' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_client_details(**kwargs)
            self.cache_set(k,r)
        #print(r)
        self.__trading_api.credentials.int_account = r["data"]["intAccount"]
        #print(f"intAccount:{self.__trading_api.credentials.int_account}")
        return r
    
    def connect(self):
        self.__trading_api.connect()
        if not self.__user_token:
            self.get_config()
        if self.__user_token:
            self.__quotecast_api = QuotecastAPI(user_token=self.__user_token)   
        session_id = self.__trading_api.connection_storage.session_id
        #print("You are now connected, with the session id :", session_id)

    def get_list_list(self):
        return self.__trading_api.get_favourites_list(raw=True)

    def create_favourite_list(self,**kwargs):
        return self.__trading_api.create_favourite_list(**kwargs)
    
    def delete_favourite_list(self,**kwargs):
        return self.__trading_api.delete_favourite_list(**kwargs)
    
    def put_favourite_list_product(self,**kwargs):
        return self.__trading_api.put_favourite_list_product(**kwargs)
    
    def get_products_config(self,**kwargs):
        k = 'get_products_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_products_config(**kwargs)
            self.cache_set(k,r)
        self.indices = {}
        for li in r['indices']:
            self.indices[li['id']] = DictObj(li)
        self.countries = {}
        for li in r['countries']:
            self.countries[li['id']] = DictObj(li)
        self.exchanges = {}
        for li in r['exchanges']:
            self.exchanges[li['id']] = DictObj(li)      
        self.stockCountries =  r['stockCountries']
        return r
     
    def get_company_ratios(self,**kwargs):
        k = 'get_company_ratios' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_company_ratios(**kwargs)
            self.cache_set(k,r)
        try:
            codes = {}
                
            if 'data' in r and 'currentRatios' in r['data'] and 'ratiosGroups' in r['data']['currentRatios']:
                for an in r['data']['currentRatios']['ratiosGroups']:
                    for i in an['items']:
                        v = i.get('value') or np.NaN  # value
                        t = i.get('type') or None # type of parameter
                        k = i.get('id') or None # name of parameter
                        m = i.get('name') or "" # meaning
                        if t == 'N' and not pd.isna(v): v = float(v)
                        #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                        if not m.__contains__(' per '): v = v * 1#000000
                        if k:
                            codes[k] = { 'meaning':m, 'value':v }

            if 'data' in r and 'forecastData' in r['data'] and 'ratios' in r['data']['forecastData']:
                for i in r['data']['forecastData']['ratios']:
                    #print(i)
                    v = i.get('value') or np.NaN  # value
                    t = i.get('type') or None # type of parameter
                    k = i.get('id') or None # name of parameter
                    m = i.get('name') or "" # meaning
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':m, 'value':v }

            if 'data' in r and 'consRecommendationTrend' in r['data'] and 'ratings' in r['data']['consRecommendationTrend']:
                for i in r['data']['consRecommendationTrend']['ratings']:
                    #print(i)
                    v = i.get('value') or np.NaN  # value
                    k = ('ratings_'+i.get('periodType')) or None # name of parameter
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':'', 'value':v }
                    
            codes['priceCurrency'] = { 'meaning':'', 'value':r['data']['currentRatios']['priceCurrency'] }
            if len(codes['priceCurrency']) <= 1:
                codes['priceCurrency'] = { 'meaning':'', 'value':r['data']['currentRatios']['currency'] }
        except:
            None
        return codes

    def get_financial_statements(self,**kwargs):
        k = 'get_financial_statements' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_financial_statements(**kwargs)
            self.cache_set(k,r)
        codes_array = []
        if r:
            try:
                for t in ('annual','interim'):
                    if t in r['data']:
                        for an in r['data'][t]:
                                endDate = datetime.strptime(an.get('endDate'), '%Y-%m-%d')#T%H:%M:%S')
                                fiscalYear = an.get('fiscalYear')
                                periodNumber = an.get('periodNumber') or 'Y'
                                codes = {}
                                for st in an['statements']:
                                    periodLength = st.get('periodLength')
                                    periodType = st.get('periodType')
                                    for i in st['items']:
                                        v = i.get('value') or np.NaN 
                                        if not pd.isna(v): v = float(v)
                                        if not i.get('meaning').__contains__(' per '): v = v * 1#000000
                                        codes[i.get('code')] = { 'meaning':i.get('meaning'), 'value':v }
                                codes_array += [ codes ]
            except:
                #print(k)
                #traceback.print_exc()
                #del self.cache_get(k)
                None
        return codes_array
    
    
    
    def get_estimates_summaries(self,**kwargs):
        k = 'get_estimates_summaries_' + str(kwargs)
        r = self.cache_get(k)
            #print("get_estimates_summaries cache hit", type(r))
        if r is None:
            r = self.__trading_api.get_estimates_summaries(**kwargs)
            #print("get_estimates_summaries cache miss", type(r))
            self.cache_set(k,r)
        return r
    
    def get_products_info(self,**kwargs):
        k = 'get_products_info' + str(kwargs)
        r = self.cache_get(k)
            #print("get_products_info cache hit", r)
        if r is None:
            r = self.__trading_api.get_products_info(**kwargs)
            #print("get_products_info cache miss", r)
            self.cache_set(k,r)
        return r

    def get_chart(self,**kwargs):
        k = 'get_chart' + str(kwargs)
        r = self.cache_get(k)
            #print("get_chart cache hit", r)
        if r is None:
            r = self.__quotecast_api.get_chart(**kwargs)
            #print("get_chart cache miss", r)
            self.cache_set(k,r)
        return r
   
    def product_search(self,**kwargs):
        k = 'product_search' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.product_search(**kwargs)
        if hasattr(r, 'products'):
            self.cache_set(k,r)
        else:
            r = None
        return r

    def get_company_profile(self,**kwargs):
        k = 'get_company_profile' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            #searching on Degiro
            r = self.__trading_api.get_company_profile(product_isin=kwargs['product_isin'], raw=kwargs['raw'])
            self.cache_set(k,r)
        
        codes = {}
        if r is not None and 'data' in r:
            r_data = r['data']
            try:
                codes['sector'] = r_data['sector']
            except:
                None
            try:
                codes['industry'] =  r_data['industry']
            except:
                None
            try:
                codes['country'] =  r_data['contacts']['COUNTRY']
            except:
                None
            try:
                codes['floatShares'] = float(r_data['shrFloating']) / 10**6
            except:
                None
   
            try:
                if 'ratios' in r_data and 'ratiosGroups' in r_data['ratios']:
                    for an in r_data['ratios']['ratiosGroups']:
                        for i in an['items']:
                            v = i.get('value') or np.NaN  # value
                            t = i.get('type') or None # type of parameter
                            k = i.get('id') or None # name of parameter
                            m = i.get('name') or "" # meaning
                            if t == 'N' and not pd.isna(v): v = float(v)
                            #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                            if not m.__contains__(' per '): v = v * 1#000000
                            if k:
                                codes[k] = { 'meaning':m, 'value':v }
                if 'forecastData' in r_data and 'ratios' in r_data['forecastData']:
                    for i in r_data['forecastData']['ratios']:
                        #print(i)
                        v = i.get('value') or np.NaN  # value
                        t = i.get('type') or None # type of parameter
                        k = i.get('id') or None # name of parameter
                        m = i.get('name') or "" # meaning
                        if t == 'N' and not pd.isna(v): v = float(v)
                        #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                        if not m.__contains__(' per '): v = v * 1#000000
                        if k:
                            codes[k] = { 'meaning':m, 'value':v }
            except:
                None
        else: 
            # searching on Yahoo! finance
            try:
                r = self.cache_get('Y_'+k)
            except:
                sym = yf.Ticker(kwargs['product_isin'])
                r = sym.info
                try:
                    r['marketCap'] /= 1000.0
                except:
                    pass
                self.cache_set('Y_'+k, r)
                print(f"OK from Yahoo {kwargs['product_isin']}")
            codes = r
        return codes


def isna(num):
    return num!= num    

def get(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and (type(d) is dict) and k in d:
        r = d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    return r

def yget(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and k in d:
        r=d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    try:
        r = float(r)
    except:
        r = str(r)
        if r == 'None' or r == '':
            r = np.NaN
    return r

def get_longtermprice(vwdIdSecondary:str):
    qrequest = Chart.Request()
    qrequest.culture = "fr-FR"
    qrequest.period = Chart.Interval.P10Y
    qrequest.requestid = "1"
    qrequest.resolution = Chart.Interval.P1M
    qrequest.series.append("ohlc:issueid:"+vwdIdSecondary)
    qrequest.tz = "Europe/Paris"
    chart = trading_api.get_chart(request=qrequest,raw=False)
    c2=ChartHelper.format_chart(chart=chart, copy=False)
    price = ChartHelper.serie_to_df(serie=chart.series[0])
    price["timestamp"] = pd.to_datetime(price["timestamp"], unit="s")
    price.set_index("timestamp", inplace=True)
    return price

    
def parallelize_dataframe(df, func, n_cores=os.cpu_count()):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def assess_map(product, eee):
    p = DictObj(dict(product))
    row = {}
    #if p.isin != 'US55302T2042': return row
    #print(product)
    try:
        company_profile = trading_api.get_company_profile(product_isin=p.isin, raw=True)
        row['symbol'] = p.symbol
        row['id'] = p.id
        row['vwdId'] = p.vwdId if hasattr(p, 'vwdId') else ''
        row['name'] = p.name
        row['sector'] =   yget(company_profile, 'sector')  
        row['industry'] = yget(company_profile, 'industry')
        if isinstance(row['industry'], str): row['industry'] = row['industry'].replace(' (NEC)', '')
        row['isin'] = p.isin
        row['country'] = yget(company_profile, 'country') 
        row['eee'] = 1 if row['country'] in eee else 0
        
        row['volume'] = get(company_profile,"VOL10DAVG") or yget(company_profile, 'volume')
        #row['volume'] = row['volume'] / 1000 if not pd.isna(row['volume'])
        row['marketCap'] = get(company_profile,"MKTCAP") or yget(company_profile, 'marketCap')  
        if not pd.isna(row['marketCap']):
            row['marketCap'] /= 1000
        row['closePrice'] = p.closePrice if hasattr(p, 'closePrice') else 0 # price shown on screen, depends on the stock exchange
        row['Cur'] = p.currency if hasattr(p, 'currency') else '' # currency shown on screen
        if row['Cur'] == 'EUR':
            row['sortCur'] = 3 # sortCur is a temp column used to determine which exchange to keep when a stock is listed on different place. Higher value means more change to be selected
        elif row['Cur'] == 'USD':
            row['sortCur'] = 1
        else:
            row['sortCur'] = 0

        codes = trading_api.get_company_ratios(product_isin=p.isin, raw=True)
        #display(codes)
        row['StmPrice'] = get(codes,"NPRICE") # price used to compute statements, ratio, etc
        row['StmCur'] = get(codes,"priceCurrency") # currency used to compute statements, ratio, etc
        #row['VOL3MAVG'] = get(codes,"VOL3MAVG")
        h = get(codes,"NHIG")
        l = get(codes,"NLOW")
        row['L%H'] = int(100*(row['StmPrice'] - l)/(h-l)) if not pd.isna(h) and not pd.isna(l) and not pd.isna(row['StmPrice']) and h>l else np.NaN
        
        h = get(codes,"PR1DAYPRC")
        l = get(codes,"PR5DAYPRC")
        t = get(codes,"PR13WKPCT")
        x = h-l if not pd.isna(h) and not pd.isna(l) else 0
        y = l-t if not pd.isna(l) and not pd.isna(t) else 0
        row['ΔPrice'] = max(x,y)
        row['β'] = get(codes,"BETA")
        row['Reco'] = get(codes,'ratings_CURR')
        row['ΔFOCF5'] = get(codes,'FOCF_AYr5CAGR')
        row['P/FCF'] = get(codes,'TTMPRFCFPS') # Price to Free Cash Flow per Share - trailing 12 months
        row['ΔREV5'] = get(codes,"REVPS5YGR")  # "Revenue/share (5 yr growth)"; -- should be > 0
        if pd.isna(row['ΔREV5']):
            row['ΔREV5'] = yget(company_profile, 'revenueGrowth')
        row['ΔNPM5'] = get(codes,"NPMTRENDGR") # "Net Profit Margin growth rate, 5 year"; -- should be > 0
        row['ΔEPS'] = get(codes,'TTMEPSCHG')   # latest "Growth rate% - EPS, TTM";
        if pd.isna(row['ΔEPS']):
            row['ΔEPS'] = yget(company_profile, 'earningsGrowth')
        row['ΔEPS3'] = get(codes,"EPSGRPCT")   # "EPS Growth rate % - , 3 year CAGR";
        row['ΔEPS5'] = get(codes,"EPSTRENDGR") # "EPS growth rate %, 5 year CAGR";

        row['fPE'] = get(codes,'ProjPE')          # forward PE
        if pd.isna(row['fPE']):
            row['fPE'] = yget(company_profile, 'forwardPE')
        row['fPS'] = get(codes,'Price2ProjSales') # forward PS -- should be 2 to 4/
        row['fPEG'] = row['fPE'] / row['ΔEPS3']  if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN    # forward PEG ratio, should be <1
        row['ROEpct'] = get(codes,'TTMROEPCT')           # Return on average equity - trailing 12 month -- should be >20%
        if pd.isna(row['ROEpct']):
            row['ROEpct'] = yget(company_profile, 'returnOnEquity')
        row['ROE5Ypct'] = get(codes,'AROE5YAVG')         # Return on average equity avg 5Y -- should be >20%
        row['P2TB'] = get(codes,'APR2TANBK')             # price to tangible book
        #row['dP2TB'] = get(codes,'BVTRENDGR')           # growth of price to tangible book, 5Y CAGR
        row['P2B'] = get(codes,'APRICE2BK')              # price to  book
        if pd.isna(row['P2B']):
            row['P2B'] = yget(company_profile, 'priceToBook')
        #row['dP2B'] = get(codes,'TanBV_AYr5CAGR')       # growth of P25B, 5Y
        row['PCF'] =  get(codes,'TTMPRCFPS')             # "Price to Cash Flow per share, near 1 idealy

        row['PE'] = get(codes,'PEINCLXOR')               #  P/E including extraordinary items - TTM - should be <50%
        if pd.isna(row['PE']):
            row['PE'] = yget(company_profile, 'trailingPE')
        row['PEG'] = row['PE'] / row['ΔEPS3'] if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN # PEG ratio, should be <1
        if pd.isna(row['PEG']):
            row['PEG'] = yget(company_profile, 'pegRatio')
        row['PS'] = get(codes,'TTMPR2REV')               #  Price to sales - trailing 12 month  -- should be between 2 to 4
        if pd.isna(row['PS']):
            row['PS'] = yget(company_profile, 'priceToSalesTrailing12Months')
            
        row['Payout'] =  get(codes,'YLD5YAVG') 
        row['%DEBT'] =  get(codes,'QTOTD2EQ') #"Total debt/total equity, percent, should be <100%
        
        if pd.isna(row['%DEBT']):
            row['%DEBT'] = yget(company_profile, 'debtToEquity')
        row['%DEBT'] = round(row['%DEBT']) if not pd.isna(row['%DEBT']) else np.NaN
        row['BV'] =  get(codes,'QBVPS') # QTANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ABVPS') # ATANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'QTANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ATANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  yget(company_profile, 'bookValue')
        row['BV'] =  round(100 * (row['BV'] - row['StmPrice']) / row['StmPrice']) if not pd.isna(row['BV']) and not pd.isna(row['StmPrice']) and row['BV'] > 0 and row['StmPrice'] > 0 else np.NaN 
        
        # book value tangible / share price, last quarter >100% is fair

        # ratio : intrinsic value from free cash flow per share / price per share - should be > 100%
        # gain of free cash flow CAGR5Y is not available on DEGIRO, so I consider "free operational" cash flow
        dFOCF =  get(codes,'FOCF_AYr5CAGR') # gain of  free operational cash flow, CAGR 5 year.
        FCFS = get(codes,'TTMFCFSHR')   # free Cash Flow per share  - trailing 12 month
        row['IV'] = FCFS*((1-((1+dFOCF/100)*0.85)**10)/(1-((1+dFOCF/100)*0.85))+10*(((1+dFOCF/100)*0.85)**10)) if not pd.isna(dFOCF) and not pd.isna(FCFS) else np.NaN
        row['IV'] = round(100*(FCFS-row['StmPrice'])/row['StmPrice']) if not pd.isna(row['StmPrice']) and row['StmPrice']>0 and not pd.isna(row['IV']) else np.NaN
        
        try:
            # 1/ EPS
            eps = get(codes,"TTMEPSINCX")   # "EPS including extraordinary items - trailing 12 month";
            # 2/ growth rate min des 2 là, ou ΔEPS5?
            gr = min(get(codes,"REVTRENDGR"), get(codes,"TanBV_AYr5CAGR"), row['ΔEPS5'])
            # 3/ projPE ou le double du précédent, min
            ppe = min (row['fPE'], 2* gr)  
            fsv = eps*((1+gr/100)**5)*ppe/2                        
            row['FV'] = round((fsv-row['StmPrice'])/row['StmPrice']*100) # 0=> stock price will double in 5 years          
        except:
            pass

        #dFOCF 	FCFS
        #BVS = get(codes,'ABVPS') #Book value (Total Equity) per share - most recent fiscal year
        #FCF = p.StmPrice / get(codes,'TTMPRFCFPS') # Price to Free Cash Flow per Share - trailing 12 months" 
        #dREV3 = get(codes,"REVGRPCT") #"Growth rate% -  Revenue, 3 year";
        #dBVS5 = get(codes,"BVTRENDGR") #"Book value per share growth rate, 5 year";
        #dTBE5 = get(codes,"TanBV_AYr5CAGR") #"Tangible Book Value, Total Equity, 5 Year CAGR";
        #dCSP5 = get(codes,"CSPTRENDGR") # "Capital Spending growth rate, 5 year";      
        #row['EV/EBITD'] = EV/EBITD if EV and EBITD and EBITD>0 else 0

         #
    except:
        print(f"Error profile {p.symbol}")
        traceback.print_exc()
    return row    

def myassess(country, stock_list, info_df):
    eee = {}
    try:
        eee = {k:1 for k in pd.read_csv("eee.csv", header=None).T.values[0]}
    except:
        pass
    try:
        if hasattr(stock_list, 'products'):
            with ThreadPoolExecutor(max_workers = os.cpu_count()) as executor:
                results = executor.map(assess_map, stock_list.products, itertools.repeat(eee))
            for row in results:
                info_df = info_df.append(row,ignore_index=True)
        else:
            print("Stock market as no product", country)
        #info_df = info_df.astype({'FV':'Int64', 'IV':'Int64', 'BV' :'Int64','%DEBT' :'Int64','L%H':'Int64','eee':'Int64'})
    except:
        traceback.print_exc()
        pass
    return info_df

class DictObj:
    def __init__(self, in_dict:dict):
        assert isinstance(in_dict, dict)
        for key, val in in_dict.items():
            if isinstance(val, (list, tuple)):
               setattr(self, key, [DictObj(x) if isinstance(x, dict) else x for x in val])
            else:
               setattr(self, key, DictObj(val) if isinstance(val, dict) else val)

 


logging.basicConfig(level=logging.INFO)

username = os.getenv("GT_DG_USERNAME") or ""
password = os.getenv("GT_DG_PASSWORD") or ""

if username == "" or password == "":
    exit(0)
    
credentials = Credentials(
    int_account=None, # updated by get_client_details()
    username=username,
    password=password,
)


trading_api = cachedApi('/home/fab/GamestonkTerminal/.cachedb',credentials)
trading_api.connect()

try:
    # get all product list, countries, marketplaces
    products_config_dict = trading_api.get_products_config(raw=True)
    # get IntAccount
    trading_api.get_client_details()

    # this is the main dataframe that will be filled up
    info_df = pd.DataFrame()
    
    # stocked are browsed from counties(, and not marketplaces). This is the most reliable to get all stocks
    for li_dict in trading_api.stockCountries:
        li = DictObj(li_dict)
        stock_country_id = li.id
        country = trading_api.countries[li.country].name
        #if country != 'TR': continue
        # it's assumed that a country has less than 10x1000 stocks, so we browse up to 10 pages and stop once we got a partial page
        for page in range(0,10):
            request_stock = ProductSearch.RequestStocks(stock_country_id=stock_country_id,limit=1000,offset=page*1000,require_total=True)
            stock_list = trading_api.product_search(request=request_stock, raw=False)
            if hasattr(stock_list, 'products'):
                size = len(stock_list.products)
                print(f"country:{country} list:All ({size} stocks for page {page+1})")
                # dowload data for all stocks in the list. It's multi-thread even though the cache system is mono-thread...
                if stock_list: info_df = myassess(country, stock_list, info_df) 
                #for p in stock_list.products:
                #    assess_map(p)
                if size != 1000: break
            else:
                break
    print(f"Number of stock entries in all exchanges: {info_df.shape[0]}")
    # we remove duplicates when a stock is listed on several exchanges. ISIN code is the index in the dataframe, however id is the key on DEGIRO favourite list.
    info_df = info_df.sort_values(by=['isin', 'sortCur', 'vwdId'], ascending = False).drop_duplicates(keep = 'first', subset = 'isin')
    info_df.set_index('isin', inplace = True)
    print(f"Number of stock entries after removing duplicates: {info_df.shape[0]}")
    info_df.drop([ 'sortCur', 'vwdId', 'StmCur', 'StmPrice'], axis=1, inplace=True)
        
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()

try:
    trading_api.logout()
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()
 

INFO:degiro_connector.trading.actions.action_connect:get_session_id:response_dict: {'isPassCodeEnabled': True, 'locale': 'fr_FR', 'redirectUrl': 'https://trader.degiro.nl/trader/', 'sessionId': 'xxxx.prod_b_126_3', 'status': 0, 'statusText': 'success'}


country:FI list:All (171 stocks for page 1)
country:ES list:All (174 stocks for page 1)
country:HU list:All (31 stocks for page 1)
country:DK list:All (243 stocks for page 1)
country:PT list:All (42 stocks for page 1)
country:SE list:All (706 stocks for page 1)
country:CH list:All (235 stocks for page 1)
country:NO list:All (266 stocks for page 1)
country:NL list:All (131 stocks for page 1)
country:AT list:All (77 stocks for page 1)
country:PL list:All (410 stocks for page 1)
country:CZ list:All (38 stocks for page 1)
country:IT list:All (291 stocks for page 1)
country:SG list:All (188 stocks for page 1)
country:TR list:All (36 stocks for page 1)
country:BE list:All (152 stocks for page 1)
country:GR list:All (163 stocks for page 1)
country:FR list:All (727 stocks for page 1)
country:IE list:All (37 stocks for page 1)
country:HK list:All (785 stocks for page 1)
country:CA list:All (1000 stocks for page 1)
country:CA list:All (269 stocks for page 2)
country:GB list:All (1000 stocks for 

In [2]:
  
def var2rank(X,Y,x):
    r = 1 # default return if x is nan
    try:
        if x == x:
            y_interp = interp1d(x=X, y=Y,fill_value=(Y[0], Y[-1]), bounds_error=False)
            r = float(y_interp(x))
    except:
        traceback.print_exc()
        None
    return r

def var2quant(x,Q,name,f, reverse): 
    Y = Q.index.to_numpy()
    Y = Y/Y[-1]+0.5
    Y = np.asarray(Y)
    if reverse:
        Y=Y[::-1]
    X=list(Q[name].to_dict().values())
    r = var2rank(X,Y,x[name])
    #print(X,Y,x[name],r)
    return r**f

def compute_rank(info_df, Q):
    f=2 
    
#    info_df['score'] =    info_df.apply(lambda x: var2quant(x,Q,'PS',f, True), axis = 1) \
#                        * info_df.apply(lambda x: var2quant(x,Q,'fPS',f, True), axis = 1) \
    info_df['score'] =    info_df.apply(lambda x: var2rank([1,2,4,5,6],[1,2*f,2*f,1.5*f,1],x['PS']), axis = 1) \
                        * info_df.apply(lambda x: var2rank([1,2,4,5,6],[1,2*f,2*f,1.5*f,1],x['fPS']), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'PEG',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'fPEG',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'%DEBT',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'Payout',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'L%H',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'Reco',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔFOCF5',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'P/FCF',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔREV5',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔNPM5',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'PE',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'fPE',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔEPS',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔEPS3',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ΔEPS5',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ROEpct',f**0.5, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'ROE5Ypct',f**0.5, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'P2B',f**0.5, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'P2TB',f**0.5, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'PCF',f, True), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'BV',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'IV',f, False), axis = 1) \
                        * info_df.apply(lambda x: var2quant(x,Q,'FV',f, False), axis = 1) 
    return info_df
    
Q = info_df.quantile(q=[0.01,0.1,0.25,0.5,0.75,0.9,0.99])
if info_df.shape[0] > os.cpu_count():
    info_df = parallelize_dataframe(info_df,partial(compute_rank, Q=Q))
else:
    compute_rank(info_df, Q=Q)

info_df=info_df.sort_values(by='score', ascending=False)
#write2csv(info_df)


In [7]:
def write2csv(df):
    now = datetime.now() # current date and time
    filename="degiro-export-"+now.strftime("%Y-%m-%d-%H-%M")+".csv (encoding utf-8)"
    filepath='.'
    fullpath=os.path.join(filepath,filename)
    print(f"Writing csv file '{fullpath}'")
    df.to_csv(fullpath, index=True, sep=str(';'), decimal=str(','),encoding='utf-8', )


def write2fav(df):
    if (df.shape[0] > 0):
        trading_api.connect()
        products_config_dict = trading_api.get_products_config(raw=True)
        trading_api.get_client_details()
        now = datetime.now()
        prefix='Screener-'
        fl = trading_api.get_list_list()
        for l in fl['data']:
            if 'name' in l and l['name'].startswith(prefix):
                trading_api.delete_favourite_list(id=l['id'])
                print(f'Deleting DEGIRO favourite list \"{l["name"]}\"')
        name=prefix+now.strftime("%Y-%m-%d-%H-%M")
        print(f'Creating DEGIRO favourite list \"{name}\"')
        favorite_list_id = trading_api.create_favourite_list(name=name)
        for p in list(df['id'][:50]):  
            # list is limited to 50 entries
            trading_api.put_favourite_list_product(id=favorite_list_id,product_id=p)
            #print(f'Adding product id {p}')
        trading_api.logout() 

        
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format


split=10**(max(1,math.floor(math.log(info_df.shape[0],10))-1))
keeptop=math.floor(split*0.986)
cropsector=max(5,math.floor(split/50))
cropindustry=max(1,math.floor(cropsector/5))
print(f"Quantiles:{split}, keeping top {keeptop}th and above, limiting to {cropsector} stocks per sector and {cropindustry} per industry.")
df = info_df
df['score'] = pd.qcut(df['score'].rank(method='first'),q=split, retbins=False, labels=False)
df=df.drop(df.index[ ( (df['score'] < keeptop) | (df['Reco'] > 2.5)| (df['L%H'] > 50) | (df['sector'] == "Financial") |(df['ΔEPS3'] <5) |(df['ΔREV5'] <10)|(df['ΔNPM5'] <5) |(df['BV'] <-15)  )]) #| (df['Reco'] > 2) | (df['industry'].str.contains('Real Estate')) |(df['IV'] <-90) |(df['FV'] <-90)  |(df['BV'] <-15)
df = df.sort_values(by=['sector','score'], ascending=False).groupby('sector').head(cropsector).sort_values(by=['industry','score'], ascending=False).groupby('industry').head(cropindustry).sort_values(by='score', ascending=False).sort_values(by=['eee','score','sector','industry','BV'], ascending=False)
df = df.fillna("—")
print('Please read the readme.rtf to get the meaning of all the columns.')
display(df)

Quantiles:1000, keeping top 986th and above, limiting to 20 stocks per sector and 4 per industry.
Please read the readme.rtf to get the meaning of all the columns.


,symbol,id,name,sector,industry,country,eee,volume,marketCap,closePrice,Cur,L%H,ΔPrice,β,Reco,ΔFOCF5,P/FCF,ΔREV5,ΔNPM5,ΔEPS,ΔEPS3,ΔEPS5,fPE,fPS,fPEG,ROEpct,ROE5Ypct,P2TB,P2B,PCF,PE,PEG,PS,Payout,%DEBT,BV,IV,FV,score
isin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PLMRCTR00015,MRC,5192098,Mercator Medical SA,Healthcare,Medical Supplies,Poland (POL),1.00,"52,590.00","766,808.90",72.02,PLN,7.00,26.17,0.85,—,—,1.76,46.36,62.16,35.64,474.73,137.05,1.59,0.40,0.00,86.99,90.29,0.74,0.72,0.95,0.95,0.00,0.36,0.13,0.00,61.00,—,575.00,999
PLEDINV00014,EDI,4683842,ED Invest SA,Capital Goods,Homebuilding,Poland (POL),1.00,"23,910.00","59,208.36",4.78,PLN,34.00,-2.14,0.61,—,26.36,3.85,14.79,42.39,-15.54,95.51,63.45,—,—,—,14.34,9.73,0.93,0.74,6.55,5.40,0.06,0.99,12.57,14.00,35.00,-68.00,—,990
SE0002158568,EAST,1177114,Eastnine AB (publ),Services,Office Real Estate Re...,Sweden (SWE),1.00,"25,480.00","2,411,771.00",107.00,SEK,20.00,34.35,0.86,—,—,18.56,12.95,29.17,96.72,66.35,45.90,11.80,9.06,0.18,21.09,12.85,0.62,0.61,—,3.19,0.05,8.83,2.20,64.00,64.00,—,182.00,986
HK3808041546,4SK,13718277,Sinotruk Hong Kong Ltd,Consumer Cyclical,Heavy Trucks,China (CHN),0.00,"3,445,720.00","33,131,920.00",1.34,EUR,18.00,7.14,1.15,2.50,141.69,7.51,28.25,45.92,94.17,31.35,101.56,4.72,0.28,0.15,23.81,15.16,0.85,0.84,2.72,3.57,0.11,0.22,4.51,8.00,30.00,-87.00,11.00,999
CA9528451052,WFC,19746991,West Fraser Timber Co...,Basic Materials,Logging & Sawmills,Canada (CAN),0.00,"853,660.00","10,649,740.00",75.00,EUR,43.00,9.37,2.02,1.80,57.06,3.00,18.31,30.77,207.12,49.30,56.12,4.38,0.91,0.09,58.16,33.79,1.76,1.14,2.41,3.07,0.06,0.81,0.84,7.00,-12.00,-68.00,124.00,999
CA1375761048,NKC,19746170,Canfor Corp,Basic Materials,Forest & Wood Products,Canada (CAN),0.00,"352,800.00","3,206,955.00",19.50,EUR,22.00,12.87,1.88,2.00,33.55,2.16,14.12,31.57,146.54,57.12,56.82,3.33,0.44,0.06,47.14,22.91,1.21,0.92,1.75,2.40,0.04,0.42,—,10.00,9.00,-54.00,26.00,999
CA2652692096,DPU,19746260,Dundee Precious Metal...,Basic Materials,Diversified Mining,Canada (CAN),0.00,"980,720.00","1,426,133.00",5.51,EUR,17.00,2.47,1.11,2.14,35.24,6.71,12.97,—,-4.27,68.45,—,6.62,1.80,0.10,21.16,10.57,1.16,1.14,3.94,5.19,0.08,1.78,—,0.00,-12.00,-84.00,19.00,999
US55305B1017,4MI,19751557,M/I Homes Inc,Capital Goods,Residential Builders ...,United States (USA),0.00,"324,260.00","1,262,185.00",40.50,EUR,1.00,24.17,2.00,1.50,—,—,17.42,25.92,61.87,53.26,50.54,2.91,0.30,0.05,27.53,18.97,0.79,0.78,3.07,3.34,0.06,0.34,—,59.00,28.00,—,-4.00,997
US1565043007,CCT,8613863,Century Communities Inc,Capital Goods,Homebuilding,United States (USA),0.00,"554,950.00","1,783,168.00",49.60,EUR,0.00,29.20,1.94,1.75,—,—,20.68,18.87,137.73,65.01,44.06,3.09,0.37,0.05,32.74,19.15,1.03,1.02,3.50,3.68,0.06,0.42,—,75.00,-2.00,—,63.00,996


In [8]:
write2csv(df)
write2fav(df)

INFO:degiro_connector.trading.actions.action_connect:get_session_id:response_dict: {'isPassCodeEnabled': True, 'locale': 'fr_FR', 'redirectUrl': 'https://trader.degiro.nl/trader/', 'sessionId': 'xxxxxx.prod_b_126_3', 'status': 0, 'statusText': 'success'}


Writing csv file './degiro-export-2022-04-02-18-08.csv (encoding utf-8)'
Deleting DEGIRO favourite list "Screener-2022-04-02-18-03"
Creating DEGIRO favourite list "Screener-2022-04-02-18-08"
